## Text summarization task of CNN dailymail dataset

In [ ]:
# Install KerasNLP, and so on
%pip install keras-nlp rouge-score tensorflow-datasets datasets

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import platform
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
import keras
import keras_nlp

# Hyperparameters
# BATCH_SIZE = 64 # p3.2xlarge ResourceExhaustedError
EMBEDDING_DIM = 64
NUM_HEADS = 8
INTERMIDIATE_DIM = 1024
VOCAB_SIZE = 15000
BATCH_SIZE = 32 # 1 does not work
NUM_EPOCHS = 100 # 1 100

In [ ]:
if platform.system() == "Darwin" and platform.processor() == "arm":
    args = {
        'trust_remote_code': False,
    }
else:
    """
    When 'trust_remote_code' is False, it does not work on AWS SageMaker.
    """
    args = {
    }
train_dataset, validation_dataset, test_dataset = tfds.load(
    'huggingface:ccdv__cnn_dailymail/3.0.0',
    split=['train', 'validation', 'test'],
    builder_kwargs=args,
)

In [ ]:
# max_input_length = max(len(row[0][0]) for ds in [preprocessed_train_dataset, preprocessed_validation_dataset, preprocessed_test_dataset] for row in ds)
# max_target_length = max(len(row[0][1]) for ds in [preprocessed_train_dataset, preprocessed_validation_dataset, preprocessed_test_dataset] for row in ds)
# max_decoder_target_length = max(len(row[1]) for ds in [preprocessed_train_dataset, preprocessed_validation_dataset, preprocessed_test_dataset] for row in ds)
# max_input_length, max_target_length, max_decoder_target_length

summarized_text_size = 256 #  1437 is the longest summarized text in dataset
min_summarized_text_size = 128

# @TODO The followings should programmatically be derived.
max_input_length = 2137
max_target_length = summarized_text_size + 1
max_decoder_target_length = summarized_text_size + 1

In [ ]:
# Wrong
# train_dataset = train_dataset.filter(lambda x: tf.strings.length(x['highlights']) <= summarized_text_size)
# validation_dataset = validation_dataset.filter(lambda x: tf.strings.length(x['highlights']) <= summarized_text_size)
# test_dataset = test_dataset.filter(lambda x: tf.strings.length(x['highlights']) <= summarized_text_size)

# for development with 1/10 entries
DEVELOPMENT = True
if DEVELOPMENT:
    if platform.system() == "Darwin" and platform.processor() == "arm":
        NUM_TAKE = 128 # 500
        train_dataset = train_dataset.take(NUM_TAKE)
        validation_dataset = validation_dataset.take(NUM_TAKE)
        test_dataset = test_dataset.take(NUM_TAKE)
    else:
        # Use 10% dataset.
        train_size = len(train_dataset) // 10 * 9
        validation_size = len(validation_dataset) // 10 * 9
        test_size = len(test_dataset) // 10 * 9
        train_dataset = train_dataset.skip(train_size)
        validation_dataset = validation_dataset.skip(validation_size)
        test_dataset = test_dataset.skip(test_size)

In [ ]:
# @see https://github.com/keras-team/keras-nlp/blob/50e041487b1d8b30b34c5fb738db3ed3406363bc/examples/machine_translation/data.py
import string
import re

strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase,
        "[%s]" % re.escape(strip_chars),
        "",
    )

vectorization_layer = keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    ragged=True,
)
# Warning: adapt, which clear the already held data inside, must be called only once.
vectorization_layer.adapt(train_dataset.concatenate(validation_dataset).concatenate(test_dataset).batch(BATCH_SIZE).map(lambda row: '[start] ' + row['article'] + ' ' + row['highlights'] + ' [end]'))

In [ ]:
input_vectorization_layer = keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    # @TODO This should be programmatically obtained
    output_sequence_length=2137,
)
target_vectorization_layer = keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=summarized_text_size + 1,
)
input_vectorization_layer.set_vocabulary(vectorization_layer.get_vocabulary())
target_vectorization_layer.set_vocabulary(vectorization_layer.get_vocabulary())

In [ ]:
vectorization_layer.vocabulary_size(), vectorization_layer.get_vocabulary(include_special_tokens=True)[0:8]

(15000, ['', '[UNK]', 'the', 'to', 'of', 'a', 'and', 'in'])

In [ ]:
# Must be False
assert not vectorization_layer(['[start]'])[0] == vectorization_layer(['start'])[0]

In [ ]:
"""
vectorization_layer(['This is a pen', 'I am a software engineer'])
#vectorization_layer(['This is a pen', 'I am a software engineer']).row_lengths().shape[0]
# 2
rows = vectorization_layer(['This is a pen', 'I am a software engineer']).row_lengths().shape[0]
vectorization_layer(['This is a pen', 'I am a software engineer']).to_tensor(shape=(rows, 10))
# .to_tensor()

RaggedTensor.to_tensor can make 0-filled Tensor
"""
def prepare_dataset(x):
    article = input_vectorization_layer(x['article'])
    highlights = tf.strings.join(['[start] ', x['highlights'], ' [end]'])
    h = vectorization_layer(highlights)
    rows = h.row_lengths().shape[0]
    sequences = h.to_tensor(shape=(rows, summarized_text_size + 1 + 1))
    highlights_decoder_input = sequences[:, :-1] # summarized_text_size - 1
    highlights_decoder_output = sequences[:, 1:] # summarized_text_size - 1
    return (
        (
            article, # encoder input
            highlights_decoder_input, # decoder input
        ),
        highlights_decoder_output, # decoder output
    )

preprocessed_train_dataset = train_dataset.batch(BATCH_SIZE).map(prepare_dataset, num_parallel_calls=tf.data.AUTOTUNE)
preprocessed_validation_dataset = validation_dataset.batch(BATCH_SIZE).map(prepare_dataset, num_parallel_calls=tf.data.AUTOTUNE)
preprocessed_test_dataset = test_dataset.batch(BATCH_SIZE).map(prepare_dataset, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
for entry in train_dataset.take(1):
    print('article: ', entry['article'].numpy())
    print('highlights: ', entry['highlights'].numpy())
for entry in preprocessed_train_dataset.take(1):
    print('encoder input: ', entry[0][0].shape, entry[0][0][:10])
    print('decoder input: ', entry[0][1].shape, entry[0][1][:10])
    print('decoder output: ', entry[1].shape, entry[1][:10])

article:  b'(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men\'s 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles. The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital. "I\'m proud of myself and I\'ll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry on until the 2016 Rio Olympics. Victory w

In [ ]:
learning_rate = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=20,
    decay_rate=0.99,
)
if platform.system() == "Darwin" and platform.processor() == "arm":
    """
    Apple Silicon mac shows tht following warning.
    WARNING:absl:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs,
    please use the legacy Keras optimizer instead,
    located at `tf.keras.optimizers.legacy.Adam`
    Therefore, keras.optimizers.legacy.Adam is used.
    """
    optimizer = keras.optimizers.legacy.RMSprop()
else:
    optimizer = keras.optimizers.RMSprop()

loss_fn = keras.losses.SparseCategoricalCrossentropy(
    reduction=keras.losses.Reduction.NONE
)
# Build model / Encoder & Decoder model
# The encoder encodes text and represents the feature vector.
# However, the decoder scheme contains this working, especially in its heads.
# That is, it is not certain whether the encoder is necessary for this task.
# There is value in the investigation.
encoder_inputs = keras.Input(
    shape=(max_input_length,),
    name="encoder_inputs"
)
encoder_embedding = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=max_input_length,
    embedding_dim=EMBEDDING_DIM,
    mask_zero=True,
)(encoder_inputs)
encoder_outputs = keras_nlp.layers.TransformerEncoder(
    num_heads=NUM_HEADS,
    intermediate_dim=INTERMIDIATE_DIM,
)(encoder_embedding)
### <temporary>
encoder_outputs = keras_nlp.layers.TransformerEncoder(
    num_heads=NUM_HEADS,
    intermediate_dim=INTERMIDIATE_DIM,
)(encoder_outputs)
### </temporary>

decoder_inputs = keras.Input(
    shape=(max_target_length,),
    name="decoder_inputs"
)
decoder_embedding = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=max_target_length,
    embedding_dim=EMBEDDING_DIM,
    mask_zero=True,
)(decoder_inputs)
decoder_outputs = keras_nlp.layers.TransformerDecoder(
    num_heads=NUM_HEADS,
    intermediate_dim=INTERMIDIATE_DIM,
)(decoder_embedding, encoder_outputs, use_causal_mask=True)
### <temporary>
decoder_outputs = keras_nlp.layers.TransformerDecoder(
    num_heads=NUM_HEADS,
    intermediate_dim=INTERMIDIATE_DIM,
)(decoder_outputs, encoder_outputs, use_causal_mask=True)
### </temporary>

outputs = keras.layers.Dense(
    VOCAB_SIZE,
    activation="softmax"
)(decoder_outputs)

model = keras.Model(
    [encoder_inputs, decoder_inputs],
    outputs,
    name="transformer_text_summarization_model",
)
# Note
# In the case that the dataset is large and the dimension is small,
# the learning rate of Adam needed to be smaller.
model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=[
        keras.metrics.SparseCategoricalAccuracy()
        # 'accuracy', #  This should not be used.
        # "loss", # This is not necessarily specified.
        # keras_nlp.metrics.RougeL()
    ]
)
model.summary()

Model: "transformer_text_summarization_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, 2137)]               0         []                            
 )                                                                                                
                                                                                                  
 token_and_position_embeddi  (None, 2137, 64)             1096768   ['encoder_inputs[0][0]']      
 ng_18 (TokenAndPositionEmb                                                                       
 edding)                                                                                          
                                                                                                  
 decoder_inputs (InputLayer  [(None, 257)]                0    

In [ ]:
# Training
history = model.fit(
    preprocessed_train_dataset,
    validation_data=preprocessed_validation_dataset,
    # Generally, 100 to 200 is used as the epoch number for generative models
    # However, because this is a prototype, the number is intentionally small.
    epochs=NUM_EPOCHS,
    # steps_per_epoch=2,
)

Epoch 1/10
      2/Unknown - 92s 41s/step - loss: 9.5984 - sparse_categorical_accuracy: 4.4593e-04

2024-06-15 00:22:31.730624: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3448698751689097448
2024-06-15 00:22:31.730947: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3449129488977265746
2024-06-15 00:22:31.730954: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1791580377556764652
2024-06-15 00:22:31.730960: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4805864642477148710
2024-06-15 00:22:31.730963: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14059500753078487928
2024-06-15 00:22:31.730972: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9912943892492438378
2024-06-15 00:22:31.730976: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv it

2/2 [==============================] - 136s 84s/step - loss: 9.5984 - sparse_categorical_accuracy: 4.4593e-04 - val_loss: 9.5383 - val_sparse_categorical_accuracy: 0.0265
Epoch 2/10


2024-06-15 00:23:14.898055: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3281590798501822680
2024-06-15 00:23:14.898094: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6357998445219267402
2024-06-15 00:23:14.898099: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6032336288843024288
2024-06-15 00:23:14.898105: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3338838992148220646
2024-06-15 00:23:14.898108: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5243714521833967148
2024-06-15 00:23:14.898115: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5326125525852836258
2024-06-15 00:23:14.898118: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv ite

2/2 [==============================] - 123s 82s/step - loss: 9.4405 - sparse_categorical_accuracy: 0.0424 - val_loss: 9.4333 - val_sparse_categorical_accuracy: 0.0324
Epoch 3/10
2/2 [==============================] - 117s 75s/step - loss: 9.2786 - sparse_categorical_accuracy: 0.0473 - val_loss: 9.3310 - val_sparse_categorical_accuracy: 0.0296
Epoch 4/10
2/2 [==============================] - 117s 74s/step - loss: 9.1165 - sparse_categorical_accuracy: 0.0482 - val_loss: 9.2298 - val_sparse_categorical_accuracy: 0.0321
Epoch 5/10
2/2 [==============================] - 113s 72s/step - loss: 8.9473 - sparse_categorical_accuracy: 0.0522 - val_loss: 9.1315 - val_sparse_categorical_accuracy: 0.0402
Epoch 6/10
2/2 [==============================] - 117s 75s/step - loss: 8.7753 - sparse_categorical_accuracy: 0.0734 - val_loss: 9.0405 - val_sparse_categorical_accuracy: 0.0635
Epoch 7/10
2/2 [==============================] - 115s 73s/step - loss: 8.6072 - sparse_categorical_accuracy: 0.0852 - va

In [ ]:
model.save('text_classification.keras')

In [ ]:
test_loss = model.evaluate(
    preprocessed_test_dataset,
)
test_loss

1/1 [==============================] - 6s 6s/step - loss: 9.2124 - sparse_categorical_accuracy: 0.0799


2024-06-14 22:57:26.276458: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8772517423569011181
2024-06-14 22:57:26.276482: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15059473345888486835
2024-06-14 22:57:26.276501: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7421055573459142177
2024-06-14 22:57:26.276508: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2722206747826129179
2024-06-14 22:57:26.276512: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13621184332750490243
2024-06-14 22:57:26.276517: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10976154340540877505
2024-06-14 22:57:26.276522: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv 

[9.212401390075684, 0.07987220585346222]

In [ ]:
def summarize(text):
    """
    Summarize text
    :param text: original text
    :return: summarized text
    """
    table = vectorization_layer.get_vocabulary()
    input_sequence = input_vectorization_layer([text])


    start_token = vectorization_layer('[start]')[0].numpy()
    end_token = vectorization_layer('[end]')[0].numpy()
    decoded_sentence = [start_token]
    for i in range(max_target_length):
        decoder_inputs = tf.convert_to_tensor(
            [decoded_sentence],
            dtype="int64",
        )
        decoder_inputs = tf.concat(
            [
                decoder_inputs,
                tf.zeros(
                    [1, max_target_length - i - 1],
                    dtype="int64",
                ),
            ],
            axis=1,
        )
        predictions = model.predict(
            [input_sequence, decoder_inputs],
            verbose=0
        )
        predicted_token = np.argmax(predictions[0, i, :])
        decoded_sentence.append(predicted_token)
        if predicted_token == end_token:
            break

    detokenized_output = []
    for token in decoded_sentence:
        detokenized_output.append(table[token])
    return " ".join(detokenized_output)
        
#     print(start_token)
#     decoder_input_sequence = vectorization_layer(['[start]'])
#     rows = decoder_input_sequence.row_lengths().shape[0]
#     decoder_input_sequence = decoder_input_sequence.to_tensor(shape=(rows, 1438))
#     print(decoder_input_sequence)

#     # article = input_vectorization_layer(x['article'])
#     # highlights = tf.strings.join(['[start] ', x['highlights'], ' [end]'])
#     # h = vectorization_layer(highlights)
#     # rows = h.row_lengths().shape[0]
#     # sequences = h.to_tensor(shape=(rows, 1439))
#     # highlights_decoder_input = sequences[:, :-1] # 1438
#     # highlights_decoder_output = sequences[:, 1:] # 1438


#     summary = []
#     for _ in range(max_target_length):
#         predictions = model.predict(
#             [input_sequence, decoder_input_sequence],
#             verbose=0
#         )
#         #np.argmax(predictions[0, i, :])
#         next_token = tf.argmax(predictions[0, -1, :])
#         print(predictions[0, -1])
#         next_word = tokenizer.index_word.get(next_token.numpy(), '[UNK]')
#         if next_word == '[end]':
#             break
#         summary.append(next_word)
#         decoder_input_sequence = tf.concat(
#             [decoder_input_sequence, tf.expand_dims([next_token], axis=-1)],
#             axis=-1
#         )
#     return ' '.join(summary)

# Sample
# sample_text = "Giant pig fell into the swimming pool at his home in Ringwood, Hampshire. It took the efforts of a team of firefighters to winch him out of the water. A wayward horse also had to be rescued from a swimming pool in Sussex."
# print("Original:", sample_text)
# print("Summary:", summarize(sample_text))
sample_text = """
(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men\'s 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles. The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital. "I\'m proud of myself and I\'ll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry on until the 2016 Rio Olympics. Victory was never seriously in doubt once he got the baton safely in hand from Ashmeade, while Gatlin and the United States third leg runner Rakieem Salaam had problems. Gatlin strayed out of his lane as he struggled to get full control of their baton and was never able to get on terms with Bolt. Earlier, Jamaica\'s women underlined their dominance in the sprint events by winning the 4x100m relay gold, anchored by Shelly-Ann Fraser-Pryce, who like Bolt was completing a triple. Their quartet recorded a championship record of 41.29 seconds, well clear of France, who crossed the line in second place in 42.73 seconds. Defending champions, the United States, were initially back in the bronze medal position after losing time on the second handover between Alexandria Anderson and English Gardner, but promoted to silver when France were subsequently disqualified for an illegal handover. The British quartet, who were initially fourth, were promoted to the bronze which eluded their men\'s team. Fraser-Pryce, like Bolt aged 26, became the first woman to achieve three golds in the 100-200 and the relay. In other final action on the last day of the championships, France\'s Teddy Tamgho became the third man to leap over 18m in the triple jump, exceeding the mark by four centimeters to take gold. Germany\'s Christina Obergfoll finally took gold at global level in the women\'s javelin after five previous silvers, while Kenya\'s Asbel Kiprop easily won a tactical men\'s 1500m final. Kiprop\'s compatriot Eunice Jepkoech Sum was a surprise winner of the women\'s 800m. Bolt\'s final dash for golden glory brought the eight-day championship to a rousing finale, but while the hosts topped the medal table from the United States there was criticism of the poor attendances in the Luzhniki Stadium. There was further concern when their pole vault gold medalist Yelena Isinbayeva made controversial remarks in support of Russia\'s new laws, which make "the propagandizing of non-traditional sexual relations among minors" a criminal offense. She later attempted to clarify her comments, but there were renewed calls by gay rights groups for a boycott of the 2014 Winter Games in Sochi, the next major sports event in Russia.
"""
print("Original:", sample_text)
print("Summary:", summarize(sample_text))

sample_text = """
Vice President Dick Cheney will serve as acting president briefly Saturday while President Bush is anesthetized for a routine colonoscopy, White House spokesman Tony Snow said Friday. Bush is scheduled to have the medical procedure, expected to take about 2 1/2 hours, at the presidential retreat at Camp David, Maryland, Snow said. Bush's last colonoscopy was in June 2002, and no abnormalities were found, Snow said. The president's doctor had recommended a repeat procedure in about five years. The procedure will be supervised by Dr. Richard Tubb and conducted by a multidisciplinary team from the National Naval Medical Center in Bethesda, Maryland, Snow said. A colonoscopy is the most sensitive test for colon cancer, rectal cancer and polyps, small clumps of cells that can become cancerous, according to the Mayo Clinic. Small polyps may be removed during the procedure. Snow said that was the case when Bush had colonoscopies before becoming president. Snow himself is undergoing chemotherapy for cancer that began in his colon and spread to his liver. Snow told reporters he had a chemo session scheduled later Friday. Watch Snow talk about Bush's procedure and his own colon cancer » . "The president wants to encourage everybody to use surveillance," Snow said. The American Cancer Society recommends that people without high-risk factors or symptoms begin getting screened for signs of colorectal cancer at age 50. E-mail to a friend .
"""
print("Original:", sample_text)
print("Summary:", summarize(sample_text))

sample_text = "There are two chickens in the garden."
print("Original:", sample_text)
print("Summary:", summarize(sample_text))

sample_text = "Two chickens fell into the swimming pool in the garden."
print("Original:", sample_text)
print("Summary:", summarize(sample_text))

Original: 
(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles. The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital. "I'm proud of myself and I'll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry on until the 2016 Rio Olympics. Victory was n

KeyboardInterrupt: 